In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
import re

In [69]:
#날짜 포맷팅 2025.08.08. 오전 5:07 -> 20250808_0507 (str)
def date_format(s): 
    pat = re.compile(r'(\d{4})\.(\d{2})\.(\d{2})\.\s*(오전|오후)\s*(\d{1,2}):(\d{2})')
    def repl(m):
        y, mo, d, ap, h, mi = m.groups()
        h = int(h)
        if ap == '오전':
            h = 0 if h == 12 else h
        else:  # 오후
            h = 12 if h == 12 else h + 12
        return f"{y}{mo}{d}_{h:02d}{mi}" 
    out = pat.sub(repl, s)
    return out 

In [70]:
#topic별 url 리스트 반환
def make_urllist(): 
    urllist=[]
    topic_txt_lnb=pd.read_csv('./naver_topicSet.csv',encoding='utf-8')
    for row in topic_txt_lnb.iloc[:,1]:
        url='https://news.naver.com/breakingnews/section%s'%str(row)
        urllist.append(url) 
    return urllist

In [71]:
#제목, 날짜, 기사 내용 크롤
def crawler(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        exit()
    soup=BeautifulSoup(html,'html.parser')
    # content 검색하여 AI, ...키워드로 삭제 저장 X
    copyright=soup.find('p',{'class':'c_text'}).text.strip()
    AI_ban=['AI'] #AI학습도 금지
    for i in AI_ban:
        if i in copyright:
            print("AI_ban")
            print(copyright)
            return 0 #종료
    raw_date=soup.find(class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').text.strip() # date
    date=date_format(raw_date)
    title=soup.find('title').text.strip() # title
    
    content=soup.find("article",id='dic_area').text.strip() # content
    return date,title,content


In [77]:
# 기사 날짜, 제목, 내용 csv로 저장
url='https://n.news.naver.com/mnews/article/029/0002974165'
crawler(url)
with open("article_data.csv","w",encoding='utf-8-sig',newline='')as file:
    csv_writer=csv.writer(file)
    csv_writer.writerow(crawler(url))